In [1]:
# Base functionality
using Plots, Measures, Statistics

# ODE Solver
using DifferentialEquations

include("../MixedMealModel.jl")

FitModelLHC (generic function with 1 method)

In [ ]:
glc, ins, trg, nfa, bwg, time = SampleData()
mglc = mean(glc, dims=1)'
mins = mean(ins, dims=1)'
mtrg = mean(trg, dims=1)'
mnfa = mean(nfa, dims=1)'
mbwg = mean(bwg);

sglc = std(glc, dims=1)'
sins = std(ins, dims=1)'
strg = std(trg, dims=1)'
snfa = std(nfa, dims=1)';

In [ ]:
m, constants, subject = ModelFromData(mglc, mins, mtrg, mnfa, mbwg)
loss, simulationfunc = LossFunction(m, mglc, mins, mtrg, mnfa, Int.(time), constants, subject)
p_init = [1.35e-2, 3.80e-3, 5.82e-1, 0.00045, 0.0713, 208.88, 0.0163, 0.0119]
p_lb = [0.005, 0, 0, 0, 0, 60., 0.005, 0]
p_ub = [0.05, 1., 10., 1., 1., 720., 0.1, 1.]
println("Initial loss for individual mean: $(loss(p_init))")

In [ ]:
res, obj = FitModelSampledLHC(loss, 100, 5, p_lb, p_ub)

In [ ]:
outputs = simulationfunc(res[argmin(obj),:][1])

glucose_data_plot = scatter(time, mglc, title="Glucose", labels="Data", ylabel = "[Glucose] (mM)", yerr=sglc, color=RGB(250/255, 86/255, 55/255))
plot!(glucose_data_plot, outputs, idxs=2, label="Model",lc=RGB(20/255,55/255,1),linewidth=2)
insulin_data_plot = scatter(time, mins, title="Insulin", ylabel = "[Insulin] uIU/ml", labels="", yerr=sins, color=RGB(250/255, 86/255, 55/255))
plot!(insulin_data_plot, outputs, idxs=4, label="",lc=RGB(20/255,55/255,1),linewidth=2)
TG_data_plot = scatter(time, mtrg, title="Triglyceride", ylabel = "[TG] mM", labels="", yerr=strg, color=RGB(250/255, 86/255, 55/255))
plot!(TG_data_plot, outputs, idxs=13, label="",lc=RGB(20/255,55/255,1),linewidth=2)
NEFA_data_plot = scatter(time, mnfa, title="NEFA", ylabel = "[NEFA] mM", labels="", yerr=snfa, color=RGB(250/255, 86/255, 55/255))
plot!(NEFA_data_plot, outputs, idxs=9, label="",lc=RGB(20/255,55/255,1), linewidth=2)


plot(glucose_data_plot, insulin_data_plot, TG_data_plot, NEFA_data_plot,  layout=4, titlefontsize=10, titlefontfamily="Helvetica Bold", fontfamily="Helvetica", 
titlelocation=:left, size=(400,400), legend=false, grid=:y,
left_margin=2mm, bottom_margin=2mm, guidefontfamily="Helvetica Bold", guidefontsize=8)
savefig("Figures/ParamEstim.svg")

## Profile Likelihood

In [1]:
using Distributed; addprocs(8, exeflags="--project")

8-element Vector{Int64}:
 2
 3
 4
 5
 6
 7
 8
 9

In [2]:

@everywhere begin
    # Benchmarking
    using BenchmarkTools
    using ProfileLikelihoodAnalysis, Optimization, SciMLSensitivity
    include("../MixedMealModel.jl")

    glc, ins, trg, nfa, bwg, time = SampleData()
    mglc = mean(glc, dims=1)'
    mins = mean(ins, dims=1)'
    mtrg = mean(trg, dims=1)'
    mnfa = mean(nfa, dims=1)'
    mbwg = mean(bwg);
    m, constants, subject = ModelFromData(mglc, mins, mtrg, mnfa, mbwg)
    loss, simulationfunc = LossFunction(m, mglc, mins, mtrg, mnfa, Int.(time), constants, subject)

    p_pla = [0.01839046060378375, 0.03364049793268846, 2.8509996539094904, 0.0005374326199918778, 0.08093925430063408, 352.4129550717545, 0.012070039791549596, 0.02379300666746938]
    p_lb = [0.005, 0, 0, 0, 0, 60., 0.005, 0]
    p_ub = [0.05, 1., 10., 1., 1., 720., 0.1, 1.]

    p = PLAProblem((x,λ) -> loss(x), p_pla)

    function runPLA(param_idx)
        PLAcompute(p, param_idx, lb=p_lb, ub=p_ub, sample_size=25, verbose=false)
    end
end

In [5]:
using ProfileLikelihoodAnalysis, Optimization, SciMLSensitivity
include("../MixedMealModel.jl")

glc, ins, trg, nfa, bwg, time = SampleData()
mglc = mean(glc, dims=1)'
mins = mean(ins, dims=1)'
mtrg = mean(trg, dims=1)'
mnfa = mean(nfa, dims=1)'
mbwg = mean(bwg);
m, constants, subject = ModelFromData(mglc, mins, mtrg, mnfa, mbwg)
loss, simulationfunc = LossFunction(m, mglc, mins, mtrg, mnfa, Int.(time), constants, subject)

p_pla = [0.01839046060378375, 0.03364049793268846, 2.8509996539094904, 0.0005374326199918778, 0.08093925430063408, 352.4129550717545, 0.012070039791549596, 0.02379300666746938]
p_lb = [0.005, 0, 0, 0, 0, 60., 0.005, 0]
p_ub = [0.05, 1., 10., 1., 1., 720., 0.1, 1.]

p = PLAProblem((x,λ) -> loss(x), p_pla)

function runPLA(param_idx; sample_size=25)
    PLAcompute(p, param_idx, lb=p_lb, ub=p_ub, sample_size=sample_size, verbose=false)
end

runPLA (generic function with 1 method)

In [8]:
res = runPLA(1; sample_size=200)

PLAResult{Float64}([0.01342503624076214, 0.014252606967932409, 0.015080177695102677, 0.015907748422272945, 0.016735319149443213, 0.01756288987661348, 0.01839046060378375, 0.019218031330954018, 0.020045602058124286, 0.020873172785294554, 0.021700743512464822, 0.02252831423963509, 0.02335588496680536, 0.024183455693975626, 0.025011026421145895], [7.813975901395169, 6.651081300828592, 5.769219735313863, 5.130148733027076, 4.702852246524596, 4.461739131539019, 4.385549174334275, 4.455971424226455, 4.6567395925213555, 4.972299838187358, 5.388352260666538, 5.891857142278912, 6.472251949065059, 7.115593902239282, 7.808883494923139], 0.01839046060378375, 4.385549174334275, 2.7326367934996627, (0.01392019322317007, 0.024186549808803946))

UndefVarError: UndefVarError: guidefontfamily not defined